<a href="https://colab.research.google.com/github/Roger-G/Data-augmentation/blob/master/SVM%20(various%20ratio).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# df_train = pd.read_csv('/content/drive/My Drive/Data/Back_trans/datasets/twitter/train.tsv', sep = '\t')
# print('info', df_train.info())
# print('value_counts', df_train.label.value_counts())
# ! wget http://nlp.stanford.edu/data/glove.6B.zip
# ! unzip glove.6B.zip

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import seaborn as sns
from torch import optim
import torch.nn.functional as F
import pickle
import csv
import torch.utils.data as Data
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') 
from sklearn.svm import LinearSVC
from tqdm import trange
import pandas as pd
import torchtext.vocab as Vocab
import collections
# with open("glove.6B.50d.txt", "rb") as lines:
#     w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
#            for line in lines}

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import svm
import os.path as path
import re
from sklearn.utils import resample
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
from sklearn.pipeline import Pipeline

In [ ]:
class Preprocessor():
    
  def __init__(self, data, max_seq_len, vocab):
    self.data = data
    self.max_seq_len = max_seq_len
    self.vocab = vocab
    self.PAD, self.BOS, self.EOS = '<pad>', '<bos>', '<eos>'
    self.all_tokens = []
    self.all_seqs = []

  def process_one_seq(self, seq_tokens):
    self.all_tokens.extend(seq_tokens) # add seq_tokens into one list, like extend the list, [..., seq_tokens].
    seq_tokens = [self.PAD] * (self.max_seq_len - len(seq_tokens)) + seq_tokens
    self.all_seqs.append(seq_tokens) # add seq_tokens become a list element, [...,[seq_tokens]].
      
  def build_data(self):
    if self.vocab:
      indices = [[self.vocab.stoi[w] for w in seq] for seq in self.all_seqs] # use its vocab_dic to represent the sentence. 
    else:
      tokens_dic = collections.Counter(self.all_tokens) 
      self.vocab = Vocab.Vocab(tokens_dic, specials=[self.PAD])
      indices = [[self.vocab.stoi[w] for w in seq] for seq in self.all_seqs] # use its vocab_dic to represent the sentence. 
    return indices
  
  @staticmethod
  def normalizeString(s):
    s = s.str.lower()
    s = s.str.replace(r"<br />",r" ")
    s = s.str.replace(r'(\W)(?=\1)', '')
    s = s.str.replace(r"([.!?])", r" \1")
    s = s.str.replace(r"[^a-zA-Z.!?]+", r" ")
    return s

  def read_data(self):
    df = self.data.copy()
    df = df.dropna()
    df['sentence'] = self.normalizeString(df['sentence'])
    df.reset_index(inplace=True, drop=True)
    k = len(df)
    target = []

    for line in range(k):
      in_seq_tokens = df['sentence'][line].split(' ')
      if len(in_seq_tokens) > self.max_seq_len - 1:
        continue
      self.process_one_seq(in_seq_tokens)
      target.append(df.label[line])

    in_data = self.build_data() # in_tokens is the list where contains every word, in_seqs is a list where its element are the sentence in French.

    return [in_data, target]
    # Data.TensorDataset(in_data, torch.tensor(target))

In [ ]:
class Loader():


  def path_setting(self, task_name, tech_name):
    abspath = '/content/drive/My Drive/Data/'
    model_path = path.join(abspath, tech_name + '/save_model')
    dataset_path = path.join(abspath, tech_name + '/datasets/{}/'.format(task_name))
    save_path = path.join(abspath, tech_name + '/aug_data/{}/'.format(task_name))
    
    return model_path, dataset_path, save_path

  @staticmethod
  def load_data(path_, FILE_NAME, ratio, tech_name):

    df_train = pd.read_csv(os.path.join(path_,'train.tsv'), sep='\t',  error_bad_lines=False)
    df_train = df_train.dropna()

    num = int(len(df_train) * ratio)
    df_train = df_train[ : num ]
    num_DA = max(df_train.label.value_counts()) - min(df_train.label.value_counts())

    if FILE_NAME != 'oversampling':
      df = pd.read_csv(os.path.join(path_, FILE_NAME + '.tsv'), sep='\t',  error_bad_lines=False)    
      df = df.dropna()
      if FILE_NAME == 'train' or tech_name == 'Back_trans':
        df = df[: num ]
    
    else:
      # 选出差额， 补到原数据上面
      df_over_samp = resample(df_train[df_train.label==0], n_samples=num_DA, random_state=100, replace=True)
      df = pd.concat([df_over_samp, df_train])

    if FILE_NAME != 'train' and FILE_NAME != 'dev' and FILE_NAME != 'oversampling' and FILE_NAME != 'test':
      try:
        df_over_samp = resample(df, n_samples=num_DA, replace = False, random_state=10) 
      except ValueError:
        df_over_samp = resample(df, n_samples=num_DA, replace = True, random_state=10) 

      df = pd.concat([df_train, df_over_samp]) 
    
    else: 
      pass

    df = df.sample(frac = 1, replace=False, random_state=102).reset_index().drop(columns = ['index'])
    return df



In [ ]:
#@title
#@



def lr_cv(splits, X, Y, pipeline, average_method):
    dev_data = pd.read_csv('/content/drive/My Drive/Data/Back_trans/aug_data/twitter/dev.tsv', sep='\t')
    dev_data.dropna(inplace=True)
    x_dev, y_dev = dev_data.sentence, dev_data.label
    kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=777)
    accuracy = []
    precision = []
    recall = []
    f1 = []
    for train, test in kfold.split(X, Y):
        lr_fit = pipeline.fit(X[train], Y[train])
        prediction = lr_fit.predict(X[test])
        scores = lr_fit.score(X[test],Y[test])
        
        accuracy.append(scores * 100)
        precision.append(precision_score(Y[test], prediction, average=average_method)*100)
        # print('first precision', precision)
        print('              negative    neutral     positive')
        print('precision:',precision_score(Y[test], prediction, average=None))
        recall.append(recall_score(Y[test], prediction, average=average_method)*100)
        print('recall:   ',recall_score(Y[test], prediction, average=None))
        f1.append(f1_score(Y[test], prediction, average=average_method)*100)
        print('f1 score: ',f1_score(Y[test], prediction, average=None))
        print('-'*50)

    prediction_dev = lr_fit.predict(x_dev)
    dev_accuracy = lr_fit.score(x_dev, y_dev)


    print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))
    print('*'* 50)
    precision = precision_score(y_dev, prediction_dev, average=average_method)
    # print('precision', precision)
    recall = recall_score(y_dev, prediction_dev, average=average_method)
    f1 = f1_score(y_dev, prediction_dev, average=average_method)
    print("accuracy: %.2f" % (dev_accuracy))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))


In [ ]:
def eval_matrix(data, pipeline, average_method, save_path):

  dev_data = pd.read_csv(path.join(save_path, 'test.tsv'), sep='\t')
  dev_data.dropna(inplace=True)
  x_dev, y_dev = dev_data.sentence, dev_data.label
  x_train, y_train = data.sentence, data.label
  lr_fit = pipeline.fit(x_train, y_train)
  prediction_dev = lr_fit.predict(x_dev)
  dev_accuracy = lr_fit.score(x_dev, y_dev)
  precision = precision_score(y_dev, prediction_dev, average=average_method)
  # print('precision', precision)
  recall = recall_score(y_dev, prediction_dev, average=average_method)
  f1 = f1_score(y_dev, prediction_dev, average=average_method)

  print("accuracy: %.4f" % (dev_accuracy))
  print("precision: %.4f" % (np.mean(precision)))
  print("recall: %.4f" % (np.mean(recall)))
  print("f1 score: %.4f" % (np.mean(f1)))


In [ ]:

# class excuter():
def pipelines(tkz, clf):

  ROS_pipeline = make_pipeline(tkz, RandomOverSampler(random_state=777),clf)
  SMOTE_pipeline = make_pipeline(tkz, SMOTE(random_state=777),clf)
  ADASYN_pipeline = make_pipeline(tkz, ADASYN(ratio='minority',random_state=777), clf)

  original_pipeline = Pipeline([
      ('vectorizer', tkz),
      ('classifier', clf)
  ])
  return ROS_pipeline, original_pipeline

def execute(test_list, ratio, task_name, tech_name, tkz, clf):

  loader = Loader()
  
  model_path, dataset_path, save_path = loader.path_setting(task_name, tech_name)
  ROS_pipeline, original_pipeline = pipelines(tkz, clf)
  for file_name in test_list:

    print('-' * 70)
    print('Start to make classification use the data : {}'.format(file_name))
    print('-' * 70)
    # DA and oversampling data"
    df_train = loader.load_data(save_path, file_name, ratio, tech_name)
    if file_name == 'oversampling':
      df_train = loader.load_data(save_path, 'train', ratio, tech_name)
      eval_matrix(df_train, ROS_pipeline, 'macro', save_path)

    else:
      eval_matrix(df_train, original_pipeline, 'macro', save_path)


## Back_trans 

### SVM  new test


In [ ]:

tvec = TfidfVectorizer(stop_words=None, max_features=2000000, ngram_range=(1, 3))

lr = LogisticRegression()
svm_clf = LinearSVC(random_state=0, tol=1e-5, dual=False)
task_name = 'sub_twitter'
tech_name = 'Back_trans'
ratios = [0.2, 0.4, 0.6, 0.8, 1]
test_list = [
            'train',
            'oversampling',
            'negative_aug',
              ]
clf = svm_clf
print('svm clf')
for ratio in ratios:
  print('*' * 50)
  print('ratio', ratio)
  
  execute(test_list, ratio, task_name, tech_name, tvec, clf)

svm clf
**************************************************
ratio 0.2
----------------------------------------------------------------------
Start to make classification use the data : train
----------------------------------------------------------------------
accuracy: 0.6490
precision: 0.5792
recall: 0.4920
f1 score: 0.4939
----------------------------------------------------------------------
Start to make classification use the data : oversampling
----------------------------------------------------------------------
accuracy: 0.6426
precision: 0.5481
recall: 0.5143
f1 score: 0.5216
----------------------------------------------------------------------
Start to make classification use the data : negative_aug
----------------------------------------------------------------------
accuracy: 0.6308
precision: 0.5375
recall: 0.5542
f1 score: 0.5430
**************************************************
ratio 0.4
----------------------------------------------------------------------
Start to

## BERT 

#### SVM (test) new

In [ ]:


%%time

tvec = TfidfVectorizer(stop_words=None, max_features=2000000, ngram_range=(1, 3))

lr = LogisticRegression()
svm_clf = LinearSVC(random_state=0, tol=1e-5, dual=False)
print('svm')

task_name = 'sub_twitter'
tech_name = 'Bert_ data augmentation-master'
clf = svm_clf
ratios = [0.2, 0.4, 0.6, 0.8, 1]
for ratio in ratios:
  if ratio == 0.2:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(unfine)_0.2',
      '3_rep_(unfine)_0.2',
      '4_rep_(unfine)_0.2',
      '5_rep_(unfine)_0.2']
    print('<'*50)
    print('ratio', ratio)
  if ratio == 0.4:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(unfine)_0.4',
      '3_rep_(unfine)_0.4',
      '4_rep_(unfine)_0.4',
      '5_rep_(unfine)_0.4']
    print('<'*50)
    print('ratio', ratio)

  if ratio == 0.6:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(unfine)_0.6',
      '3_rep_(unfine)_0.6',
      '4_rep_(unfine)_0.6',
      '5_rep_(unfine)_0.6']
    print('<'*50)
    print('ratio', ratio)

  if ratio == 0.8:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(unfine)_0.8',
      '3_rep_(unfine)_0.8',
      '4_rep_(unfine)_0.8',
      '5_rep_(unfine)_0.8']
    print('<'*50)
    print('ratio', ratio)

  if ratio == 1:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(unfine)_1',
      '3_rep_(unfine)_1',
      '4_rep_(unfine)_1',
      '5_rep_(unfine)_1']
    print('<'*50)
    print('ratio', ratio)

  execute(test_list, ratio, task_name, tech_name, tvec, clf)




svm
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
ratio 0.2
----------------------------------------------------------------------
Start to make classification use the data : train
----------------------------------------------------------------------
accuracy: 0.6490
precision: 0.5792
recall: 0.4920
f1 score: 0.4939
----------------------------------------------------------------------
Start to make classification use the data : oversampling
----------------------------------------------------------------------
accuracy: 0.6426
precision: 0.5481
recall: 0.5143
f1 score: 0.5216
----------------------------------------------------------------------
Start to make classification use the data : 2_rep_(unfine)_0.2
----------------------------------------------------------------------
accuracy: 0.6346
precision: 0.5367
recall: 0.5368
f1 score: 0.5355
----------------------------------------------------------------------
Start to make classification use the data : 3_rep_(unfine)_0.2
----

## Fine-tuned BERT

### SVM

In [ ]:


%%time

tvec = TfidfVectorizer(stop_words=None, max_features=2000000, ngram_range=(1, 3))

lr = LogisticRegression()
svm_clf = LinearSVC(random_state=0, tol=1e-5, dual=False)
print('svm')

task_name = 'sub_twitter'
tech_name = 'Bert_ data augmentation-master'
clf = svm_clf
ratios = [0.2, 0.4, 0.6, 0.8, 1]
for ratio in ratios:
  if ratio == 0.2:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(tuned)_0.2',
      '3_rep_(tuned)_0.2',
      '4_rep_(tuned)_0.2',
      '5_rep_(tuned)_0.2']
    print('<'*50)
    # print('ratio', ratio)
  if ratio == 0.4:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(tuned)_0.4',
      '3_rep_(tuned)_0.4',
      '4_rep_(tuned)_0.4',
      '5_rep_(tuned)_0.4']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.6:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(tuned)_0.6',
      '3_rep_(tuned)_0.6',
      '4_rep_(tuned)_0.6',
      '5_rep_(tuned)_0.6']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.8:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(tuned)_0.8',
      '3_rep_(tuned)_0.8',
      '4_rep_(tuned)_0.8',
      '5_rep_(tuned)_0.8']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 1:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(tuned)_1',
      '3_rep_(tuned)_1',
      '4_rep_(tuned)_1',
      '5_rep_(tuned)_1']

    print('<'*50)
    
  execute(test_list, ratio, task_name, tech_name, tvec, clf)




svm
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
----------------------------------------------------------------------
Start to make classification use the data : train
----------------------------------------------------------------------
accuracy: 0.6490
precision: 0.5792
recall: 0.4920
f1 score: 0.4939
----------------------------------------------------------------------
Start to make classification use the data : oversampling
----------------------------------------------------------------------
accuracy: 0.6426
precision: 0.5481
recall: 0.5143
f1 score: 0.5216
----------------------------------------------------------------------
Start to make classification use the data : 2_rep_(tuned)_0.2
----------------------------------------------------------------------
accuracy: 0.6337
precision: 0.5329
recall: 0.5293
f1 score: 0.5296
----------------------------------------------------------------------
Start to make classification use the data : 3_rep_(tuned)_0.2
----------------

## Fasttext

### SVM test new

In [ ]:





tvec = TfidfVectorizer(stop_words=None, max_features=1000000, ngram_range=(1, 3))

lr = LogisticRegression()
svm_clf = svm.SVC(kernel = 'linear')
clf = svm_clf
print('lr')

task_name = 'sub_twitter'
tech_name = 'Fasttext_augmentation'
test_list = [
      'train',
      'oversampling',
      '2_rep_(tuned)',
      '3_rep_(tuned)',
      # '4_rep_(tuned)',
      
      ]
ratios = [0.2, 0.4, 0.6, 0.8, 1]
for ratio in ratios:
  print('ratio', ratio)
  if ratio == 0.2:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(tuned)_0.2',
      '3_rep_(tuned)_0.2',
      '4_rep_(tuned)_0.2',
      '5_rep_(tuned)_0.2']
    print('<'*50)
    # print('ratio', ratio)
  if ratio == 0.4:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(tuned)_0.4',
      '3_rep_(tuned)_0.4',
      '4_rep_(tuned)_0.4',
      '5_rep_(tuned)_0.4']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.6:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(tuned)_0.6',
      '3_rep_(tuned)_0.6',
      '4_rep_(tuned)_0.6',
      '5_rep_(tuned)_0.6']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.8:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(tuned)_0.8',
      '3_rep_(tuned)_0.8',
      '4_rep_(tuned)_0.8',
      '5_rep_(tuned)_0.8']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 1:
    test_list = ['train',
                 'oversampling',
                 '2_rep_(tuned)_1',
      '3_rep_(tuned)_1',
      '4_rep_(tuned)_1',
      '5_rep_(tuned)_1']
    print('<'*50)
    # print('ratio', ratio)


  
  execute(test_list, ratio, task_name, tech_name, tvec, clf)


lr
ratio 0.2
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
----------------------------------------------------------------------
Start to make classification use the data : train
----------------------------------------------------------------------
accuracy: 0.6501
precision: 0.5820
recall: 0.4774
f1 score: 0.4683
----------------------------------------------------------------------
Start to make classification use the data : oversampling
----------------------------------------------------------------------
accuracy: 0.6438
precision: 0.5527
recall: 0.5018
f1 score: 0.5095
----------------------------------------------------------------------
Start to make classification use the data : 2_rep_(tuned)_0.2
----------------------------------------------------------------------


FileNotFoundError: ignored

## EDA

### SVM (test, new)

In [ ]:
%%time
clf = svm_clf
test_list = [
      # 'train',
      # 'oversampling',
      '2_del',
      '3_del',
      '4_del',
      # '5_del',
      
      ]
task_name = 'sub_twitter'
tech_name = 'EDA'
ratios = [0.2, 0.4, 0.6, 0.8, 1]
for ratio in ratios:
  print('ratio', ratio)
  if ratio == 0.2:
    test_list = ['train',
                 'oversampling',
                 '2_del_0.2',
      '3_del_0.2',
      '4_del_0.2',
      ]
    print('<'*50)
    # print('ratio', ratio)
  if ratio == 0.4:
    test_list = ['train',
                 'oversampling',
                 '2_del_0.4',
      '3_del_0.4',
      '4_del_0.4',
      ]
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.6:
    test_list = ['train',
                 'oversampling',
                 '2_del_0.6',
      '3_del_0.6',
      '4_del_0.6',
      ]
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.8:
    test_list = ['train',
                 'oversampling',
                 '2_del_0.8',
      '3_del_0.8',
      '4_del_0.8',
      ]
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 1:
    test_list = ['train',
                 'oversampling',
                 '2_del_1',
      '3_del_1',
      '4_del_1',
      ]

    print('<'*50)
  execute(test_list, ratio, task_name, tech_name, tvec, clf)

ratio 0.2
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
----------------------------------------------------------------------
Start to make classification use the data : train
----------------------------------------------------------------------
accuracy: 0.6501
precision: 0.5820
recall: 0.4774
f1 score: 0.4683
----------------------------------------------------------------------
Start to make classification use the data : oversampling
----------------------------------------------------------------------
accuracy: 0.6438
precision: 0.5527
recall: 0.5018
f1 score: 0.5095
----------------------------------------------------------------------
Start to make classification use the data : 2_del_0.2
----------------------------------------------------------------------
accuracy: 0.6347
precision: 0.5321
recall: 0.5250
f1 score: 0.5279
----------------------------------------------------------------------
Start to make classification use the data : 3_del_0.2
--------------------------